In [2]:
%pip install -r requirements.txt
%pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [1]:
#use logger
import logging
import time
import utils.config as config
import utils.io_utils as io_utils
import tracemalloc
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

tracemalloc.start()

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

async def main():
    try:
        start_time = time.time()
        await io_utils.extract_EaglePack_and_process(config.input_path)
        end_time = time.time()
        logger.info(f"Total processing time: {end_time - start_time:.2f} seconds")
    finally:
        if config.use_LLM:
            await config.llm_processor.stop()
    logger.info("Processing completed")

if __name__ == "__main__":
    asyncio.run(main())
else:
    # For environments like Jupyter Notebooks, where __name__ != "__main__"
    asyncio.create_task(main())

Not using LLM


RuntimeError: asyncio.run() cannot be called from a running event loop